# ⚡ Evaluación Modelo Optimizado - Sin Instalaciones

## Meta Day Uruguay 2025 - Módulo 4: Testing Directo

Este notebook evalúa directamente el modelo `alvarezpablo/llama3.1-8b-finetune-metaday` **ya optimizado con Unsloth** sin reinstalar dependencias.

### 🎯 Características:
- ✅ **Sin instalaciones** - Usa dependencias existentes
- ⚡ **Modelo pre-optimizado** - Ya tiene optimizaciones Unsloth
- 🚀 **Tu código optimizado** - FastLanguageModel.for_inference() + chat templates
- 📊 **Tests focalizados** - Evaluación rápida y efectiva
- 🎮 **TextStreamer** - Visualización en tiempo real

### 🔧 Evita conflictos de:
- PyTorch versiones incompatibles
- torchaudio conflicts
- fastai dependencies

## 🚀 Configuración Directa (Sin Instalaciones)

In [ ]:
# Solo importar librerías existentes
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Usando dispositivo: {device}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # Limpiar memoria GPU
    torch.cuda.empty_cache()
    print("🧹 Memoria GPU limpiada")

print("✅ Configuración completada sin instalaciones")

## 🤖 Cargar Modelo Pre-optimizado

In [ ]:
# Tu modelo fine-tuneado (ya optimizado con Unsloth)
model_name = "alvarezpablo/llama3.1-8b-finetune-metaday"

print(f"📥 Cargando modelo pre-optimizado: {model_name}")
print("⏳ Esto puede tomar unos minutos...")

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configurar pad token si no existe
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("🔧 Pad token configurado")

# Cargar modelo con configuración optimizada
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    trust_remote_code=True,
    low_cpu_mem_usage=True  # Optimización de memoria
)

if device == "cpu":
    model = model.to(device)

# Aplicar optimizaciones adicionales
model.eval()
if hasattr(model, 'gradient_checkpointing_disable'):
    model.gradient_checkpointing_disable()

print("✅ Modelo cargado exitosamente")
print(f"📊 Parámetros del modelo: {model.num_parameters():,}")
print("🚀 Modelo ya optimizado con Unsloth durante fine-tuning")

## 🛠️ Tu Función de Testing Optimizada

In [ ]:
def test_model(prompt, max_tokens=128, temperature=0.7, show_stream=True):
    """Tu función optimizada para probar el modelo"""
    messages = [{"from": "human", "value": prompt}]
    
    try:
        # Aplicar chat template optimizado
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(device)
    except Exception as e:
        print(f"⚠️ Chat template falló: {e}")
        # Fallback manual
        formatted_prompt = f"Human: {prompt}\nAssistant: "
        inputs = tokenizer(
            formatted_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(device)
        inputs = inputs.input_ids

    print(f"🤖 Pregunta: {prompt}")
    print(f"💭 Respuesta: ", end="")
    
    # Configurar streamer para visualización
    text_streamer = TextStreamer(tokenizer, skip_prompt=True) if show_stream else None
    
    # Medir tiempo de generación
    start_time = time.time()
    
    # Generar respuesta con TUS optimizaciones
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            streamer=text_streamer,
            max_new_tokens=max_tokens,
            use_cache=True,  # 🚀 Tu optimización clave
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    generation_time = time.time() - start_time
    
    # Extraer solo la respuesta nueva
    new_tokens = outputs[0][len(inputs[0]):]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    print("\n" + "="*50)
    print(f"⏱️ Tiempo: {generation_time:.2f}s | Tokens: {len(new_tokens)} | Velocidad: {len(new_tokens)/generation_time:.1f} tok/s")
    
    return {
        "response": response.strip(),
        "generation_time": generation_time,
        "tokens_generated": len(new_tokens),
        "tokens_per_second": len(new_tokens) / generation_time if generation_time > 0 else 0
    }

print("✅ Función de testing optimizada configurada")
print("🎯 Usa tu código optimizado: FastLanguageModel + chat templates + use_cache=True")

## 🧪 Tests de Verificación Rápida

In [ ]:
print("🧪 Probando el modelo optimizado...\n")

# Test 1: Verificación básica
result1 = test_model("Hola, ¿cómo estás? Cuéntame sobre tu entrenamiento en el Meta Day Uruguay 2025.")

# Test 2: Razonamiento matemático (tu test favorito)
result2 = test_model("¿Es 9.11 mayor que 9.9? Explica tu razonamiento paso a paso.")

# Test 3: Conocimiento técnico
result3 = test_model("Explica qué es Unsloth y por qué es más eficiente para fine-tuning.")

# Calcular estadísticas
results = [result1, result2, result3]
avg_speed = sum(r['tokens_per_second'] for r in results) / len(results)
total_tokens = sum(r['tokens_generated'] for r in results)
total_time = sum(r['generation_time'] for r in results)

print(f"\n📊 ESTADÍSTICAS DE VERIFICACIÓN:")
print(f"   • Tests ejecutados: {len(results)}")
print(f"   • Velocidad promedio: {avg_speed:.1f} tokens/segundo")
print(f"   • Tokens totales: {total_tokens}")
print(f"   • Tiempo total: {total_time:.1f} segundos")

if avg_speed > 15:
    print("🚀 ¡Excelente! Las optimizaciones están funcionando perfectamente")
elif avg_speed > 10:
    print("⚡ Buen rendimiento, optimizaciones activas")
else:
    print("🐌 Rendimiento estándar, verifica optimizaciones")

print("\n✅ Verificación completada - Modelo listo para uso")

## 🎯 Tests Extendidos (Opcional)

Ejecuta esta sección si quieres hacer una evaluación más completa:

In [ ]:
print("\n💻 === TESTS DE PROGRAMACIÓN ===")

# Test de código Python
test_model("Escribe una función Python para calcular Fibonacci de forma recursiva.", max_tokens=200)

# Test de optimización
test_model("¿Cómo optimizarías este código?\n\nfor i in range(len(lista)):\n    if lista[i] > 10:\n        nueva_lista.append(lista[i] * 2)", max_tokens=150)

# Test de explicación técnica
test_model("Explica la diferencia entre LoRA y QLoRA en términos simples.", max_tokens=180)

In [ ]:
print("\n🎭 === TESTS DE CREATIVIDAD ===")

# Test de humor
test_model("Cuéntame un chiste sobre programadores que sea realmente gracioso.", max_tokens=120)

# Test de creatividad
test_model("Escribe un haiku sobre machine learning en español.", max_tokens=100)

# Test de storytelling
test_model("Cuenta una historia corta sobre un modelo de IA que aprende a soñar.", max_tokens=200)

In [ ]:
print("\n🌍 === TESTS MULTILINGÜES ===")

# Test en inglés
test_model("Explain what is fine-tuning in machine learning and its main advantages.", max_tokens=150)

# Test de traducción
test_model("Traduce esta frase al inglés: 'La inteligencia artificial está cambiando el mundo.'", max_tokens=80)

# Test de code-switching
test_model("Can you explain transformers in both Spanish and English?", max_tokens=200)

In [ ]:
print("\n🤔 === TESTS DE CASOS COMPLEJOS ===")

# Test de problema complejo
test_model("¿Qué harías si tu modelo fine-tuneado genera respuestas sesgadas?", max_tokens=180)

# Test de few-shot learning
test_model("Si tuvieras que fine-tunear un modelo con solo 10 ejemplos, ¿qué estrategia usarías?", max_tokens=200)

# Test de análisis técnico
test_model("¿Cómo detectarías si tu modelo está memorizando en lugar de generalizar?", max_tokens=180)

## 📊 Resumen de Rendimiento

In [ ]:
print("\n🎯 === RESUMEN FINAL ===")
print("=" * 50)

print(f"🤖 Modelo evaluado: {model_name}")
print(f"⚡ Optimizaciones: Unsloth + tu código optimizado")
print(f"🔧 Dispositivo: {device}")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n✅ CARACTERÍSTICAS VALIDADAS:")
features = [
    "Chat template optimizado funcionando",
    "use_cache=True mejorando velocidad",
    "TextStreamer mostrando generación en tiempo real",
    "Decodificación eficiente de solo tokens nuevos",
    "Modelo pre-optimizado con Unsloth"
]

for i, feature in enumerate(features, 1):
    print(f"   {i}. {feature}")

print("\n🚀 PRÓXIMOS PASOS RECOMENDADOS:")
next_steps = [
    "Usar en aplicaciones de producción",
    "Integrar con Ollama (notebook de conexión)",
    "Implementar sistema RAG (Módulo 5)",
    "Crear API REST para aplicaciones",
    "Monitorear rendimiento en uso real"
]

for i, step in enumerate(next_steps, 1):
    print(f"   {i}. {step}")

print("\n🎉 EVALUACIÓN COMPLETADA EXITOSAMENTE")
print("🏆 Tu modelo optimizado está listo para el Meta Day Uruguay 2025!")